In [17]:
def fin_positive_cycle(graph, weights):
    V = list(graph.keys())
    num_vertices = len(V)
    
    # Initialize the istance matrix
    d = [[float('inf')] * num_vertices for _ in range(num_vertices)]
    for i in range(num_vertices):
        d[i][i] = 0
        
    # Initialize the next vertex matrix
    next_vertex = [[None] * num_vertices for _ in range(num_vertices)]
    
    # Initialize the weights in the distance matrix
    for u in range(num_vertices):
        for v in range(num_vertices):
            if (u+1, v+1) in weights:
                d[u][v] = -weights[(u+1, v+1)]
                next_vertex[u][v] = v
    
    # Perform the Floyd-Warshall algorithm
    for k in range(num_vertices):
        for i in range(num_vertices):
            for j in range(num_vertices):
                if d[i][k] + d[k][j] < d[i][j]:
                    d[i][j] = d[i][k] + d[k][j]
                    next_vertex[i][j] = next_vertex[i][k]
    
    # Find a positive weight cycle
    for v in range(num_vertices):
        if d[v][v] < 0:
            cycle = []
            u = v
            while True:
                u = next_vertex[u][v]
                cycle.append((u+1, v+1))
                if u == v:
                    break
            return cycle
    
    return None


In [18]:
import gurobipy as gp
from gurobipy import GRB
import csv

weights = {
    (1, 1): 0.5,
    (1, 2): 0.6,
    (2, 1): 0.8,
    (2, 2): 0.4,
    (3, 4): 0.4,
    (4, 3): 0.5,
    (4, 5): 0.5,
    (5, 3): 0.7,
    (6, 5): 0.9,
    (6, 6): 0.8
}
weights = {}
head = True
with open('normal.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=';')
    for row in reader:
        if head == True:
            head = False
            nb_pairs = int(row[0])
            nb_links = int(row[1])
            M = int(row[2])
            continue
        #weights[('d' + str(int(row[0]))), ('p'+str(int(row[1])))] = float(row[2])
        weights[(int(row[0])), (int(row[1]))] = float(row[2])

print(weights)
print("size of weights: ", len(weights))

donors = set()
patients = set()
for d, p in weights:
    donors.add(d)
    patients.add(p)

# Create a new model
model = gp.Model("KidneyExchange")

# Create decision variables
x = model.addVars(donors, patients, vtype=GRB.BINARY, name="x") #meaning x[d, p] = 1 if donor d is assigned to patient p, 0 otherwise
print(x)
# complete x with 0s
for d in range(nb_pairs):
    for p in range(nb_pairs):
        if (d, p) not in weights:
            x[d, p] = model.addVar(vtype=GRB.BINARY, name=f"x[{d},{p}]")
            weights[(d, p)] = 0
            x[d, p].start = 0
print(f"new x: {x}")
# Set objective function
obj = gp.quicksum(weights[d, p] * x[d, p] for d, p in weights)
model.setObjective(obj, GRB.MAXIMIZE)

# Add assignment constraints (each donor can be assigned to at most one patient)
for d in donors:
    model.addConstr(gp.quicksum(x[d, p] for p in patients) <= 1)

# Add assignment constraints (each patient can receive at most one donor)
for p in patients:
    model.addConstr(gp.quicksum(x[d, p] for d in donors) <= 1)

# Optimize the model
model.setParam("OutputFlag", 0)
model.optimize()
print(x)
# Print the optimal solution
if model.status == GRB.OPTIMAL:
    print(f"size of donors: {len(donors)}, size of patients: {len(patients)}")
    print(f"Optimal solution found with objective: {model.objVal}")
    print("Optimal solution:")
    for d in donors:
        for p in patients:
           if x[d, p].x > 0:
                    print(f"Assign donor {d} to patient {p}")
else:
    print("No solution found.")


{(0, 0): 0.8, (0, 1): 0.4, (0, 3): 0.4, (0, 4): 0.8, (2, 0): 0.4, (2, 2): 0.7, (2, 6): 0.4, (3, 0): 0.4, (3, 3): 0.9, (3, 13): 0.4, (3, 14): 0.4, (4, 0): 0.6, (4, 3): 0.6, (4, 5): 0.5, (4, 15): 0.5, (4, 16): 0.5, (5, 5): 0.7, (5, 7): 0.9, (5, 13): 0.9, (5, 14): 0.9, (6, 3): 0.6, (6, 5): 0.5, (6, 6): 0.6, (6, 19): 0.7, (7, 5): 0.9, (7, 9): 0.5, (8, 2): 0.5, (8, 5): 0.6, (8, 6): 0.9, (9, 5): 0.5, (9, 9): 0.8, (9, 16): 0.6, (10, 10): 0.7, (10, 11): 0.8, (11, 0): 0.9, (11, 14): 0.9, (11, 19): 0.4, (12, 0): 0.7, (12, 6): 0.5, (12, 10): 0.4, (12, 13): 0.6, (12, 14): 0.5, (12, 15): 0.9, (13, 12): 0.4, (13, 13): 0.9, (13, 14): 0.9, (14, 5): 0.5, (14, 12): 0.4, (15, 4): 0.7, (16, 12): 0.6, (16, 15): 0.5, (16, 16): 0.7, (16, 17): 0.5, (17, 17): 0.5, (18, 7): 0.5, (18, 8): 0.7, (18, 14): 0.5, (18, 15): 0.5, (18, 17): 0.9, (18, 19): 0.4, (19, 1): 0.7, (19, 15): 0.4, (19, 16): 0.9, (19, 19): 0.8}
size of weights:  64
{(0, 0): <gurobi.Var *Awaiting Model Update*>, (0, 1): <gurobi.Var *Awaiting Model

In [19]:
def convert_to_tuples(lst):
    if len(lst) < 2:
        return [(lst[0], lst[0])]
    
    result = [(lst[i], lst[i+1]) for i in range(len(lst) - 1)]
    if lst[-1] == lst[0]:
        result.append((lst[-1], lst[0]))
    if result[-1][0] == result[-1][1]:
        result.pop()
    return result



In [20]:
def kydneys(donors, patients, weights, M, C, nb_pairs ):

    # Create a new model
    model = gp.Model("KidneyExchange")

    # Create decision variables
    x = model.addVars(donors, patients, vtype=GRB.BINARY, name="x") #meaning x[d, p] = 1 if donor d is assigned to patient p, 0 otherwise

    # complete x with 0s
    for d in range(nb_pairs):
        for p in range(nb_pairs):
            if (d, p) not in weights:
                x[d, p] = model.addVar(vtype=GRB.BINARY, name=f"x[{d},{p}]")
                weights[(d, p)] = 0
                x[d, p].start = 0
    # Set objective function
    obj = gp.quicksum(weights[d, p] * x[d, p] for d, p in weights)
    model.setObjective(obj, GRB.MAXIMIZE)

    
    
    # Add assignment constraints (each donor can be assigned to at most one patient)
    for d in donors:
        model.addConstr(gp.quicksum(x[d, p] for p in patients) <= 1)

    # Add assignment constraints (each patient can receive at most one donor)
    for p in patients:
        model.addConstr(gp.quicksum(x[d, p] for d in donors) <= 1)

    # Add constraints ∑(u,v)∈c x[u][v] ⩽ |c| − 1 for each cycle c in C to avoid cycles performing strictly more than M exchanges
    #print(x)
    for cycle in C:
        cycle = convert_to_tuples(cycle)
        model.addConstr(gp.quicksum(x[int(u), int(v)] for u, v in cycle) <= len(cycle) - 1)
    

    # Optimize the model
    model.setParam("OutputFlag", 0)
    model.optimize()
    solution = []
    # Print the optimal solution
    if model.status == GRB.OPTIMAL:
        for d in donors:
            for p in patients:
                if x[d, p].x > 0:
                    solution.append((str(d), str(p)))
    else:
        print("No solution found.")
    return solution



In [21]:
def form_chain_and_cycles(edges):
    chains = []
    cycles = []
    nodes = set()
    
    # Create a dictionary to store the edges
    edge_dict = {}
    for edge in edges:
        giver, receiver = edge
        edge_dict[giver] = receiver
        nodes.add(giver)
        nodes.add(receiver)
    
    # Find the starting nodes for the chains
    start_nodes = nodes.difference(edge_dict.values())
    
    # Traverse the edges to form the chains
    for start_node in start_nodes:
        chain = []
        current_node = start_node
        while current_node is not None:
            chain.append(current_node)
            current_node = edge_dict.get(current_node)
        chains.append(chain)
    
    # Find cycles in the edges
    visited = set()
    for node in nodes:
        if node not in visited:
            cycle = find_cycle(node, [], edge_dict, visited)
            if cycle:
                cycle.append(cycle[0])
                cycles.append(cycle)
    final = []
    for chain in chains:
        final.append(chain)
    for cycle in cycles:
        final.append(cycle)
    return final

def find_cycle(node, path, edge_dict, visited):
    if node in path:
        cycle_start = path.index(node)
        return path[cycle_start:]
    
    visited.add(node)
    path.append(node)
    next_node = edge_dict.get(node)
    if next_node:
        return find_cycle(next_node, path, edge_dict, visited)
    
    return None


In [25]:
import gurobipy as gp
from gurobipy import GRB
import csv
from copy import deepcopy

weights = {
    (1, 1): 0.5,
    (1, 2): 0.6,
    (2, 1): 0.8,
    (2, 2): 0.4,
    (3, 4): 0.4,
    (4, 3): 0.5,
    (4, 5): 0.5,
    (5, 3): 0.7,
    (6, 5): 0.9,
    (6, 6): 0.8
}
weights = {}
head = True
with open('large.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=';')
    for row in reader:
        if head == True:
            head = False
            nb_pairs = int(row[0])
            nb_links = int(row[1])
            M = int(row[2])
            continue
        weights[(int(row[0])), (int(row[1]))] = float(row[2])

donors = set()
patients = set()
for d, p in weights:
    donors.add(d)
    patients.add(p)

copy_weights = deepcopy(weights)
copy_donors = deepcopy(donors)
copy_patients = deepcopy(patients)
C = [] # list of cycles
#solution = kydneys(copy_donors, copy_patients, copy_weights, M, C)
#print(solution)
while True:
    inside = False
    solution = kydneys(copy_donors, copy_patients, copy_weights, M, C, nb_pairs)
    # Check solution's cycles exceed the maximum cycle length
    for cycle in form_chain_and_cycles(solution):
        if len(cycle)-1 > M:
            C.append(cycle)
            inside = True
            copy_weights = deepcopy(weights)
            copy_donors = deepcopy(donors)
            copy_patients = deepcopy(patients)
    if not inside:
        break
print("FINAL SOLUTION")
# print the solution objective value
print(f"Optimal solution found with objective: {model.objVal}")
print(solution)
for d, p in solution:
    print(f"Assign donor {d} to patient {p}")
print( "cycle de la solution: ", form_chain_and_cycles(solution))

FINAL SOLUTION
Optimal solution found with objective: 13.8
[('0', '9'), ('1', '34'), ('2', '7'), ('3', '33'), ('4', '8'), ('5', '5'), ('6', '21'), ('8', '2'), ('9', '15'), ('10', '11'), ('11', '18'), ('12', '6'), ('13', '16'), ('14', '17'), ('15', '22'), ('16', '19'), ('17', '12'), ('18', '10'), ('19', '13'), ('22', '25'), ('23', '28'), ('24', '29'), ('26', '20'), ('27', '26'), ('28', '27'), ('30', '38'), ('31', '39'), ('32', '3'), ('33', '30'), ('34', '36'), ('36', '35'), ('37', '31'), ('39', '24')]
Assign donor 0 to patient 9
Assign donor 1 to patient 34
Assign donor 2 to patient 7
Assign donor 3 to patient 33
Assign donor 4 to patient 8
Assign donor 5 to patient 5
Assign donor 6 to patient 21
Assign donor 8 to patient 2
Assign donor 9 to patient 15
Assign donor 10 to patient 11
Assign donor 11 to patient 18
Assign donor 12 to patient 6
Assign donor 13 to patient 16
Assign donor 14 to patient 17
Assign donor 15 to patient 22
Assign donor 16 to patient 19
Assign donor 17 to patient 12